# Importación de librerias
Ademas de cargar el DataSet ('train.csv')


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

# Cargar dataset
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Seleccionar las columnas de entrada y dalida, convertir columnas a tipo 'category', ademas de verificar valores faltantes en estas categorias

In [2]:
# Seleccionar las columnas de entrada (features) y la de salida (target)
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']
target = 'Survived'

X = df[features]
y = df[target]

# Convertir columnas categóricas a tipo 'category'
X['Sex'] = X['Sex'].astype('category')
X['Embarked'] = X['Embarked'].astype('category')

# Verificar valores faltantes
print("Valores nulos por columna:")
print(X.isnull().sum())

Valores nulos por columna:
Pclass        0
Sex           0
Age         177
Fare          0
Embarked      2
dtype: int64


/tmp/ipykernel_12737/3806810457.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Sex'] = X['Sex'].astype('category')
/tmp/ipykernel_12737/3806810457.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Embarked'] = X['Embarked'].astype('category')


## Rellenar valores faltantes (categoricos y numericos), combinar ambos transformadores y por ultimo dividir el dataset en entrenamiento y prueba.

In [3]:
# Dividir columnas numéricas y categóricas
numeric_features = ['Age', 'Fare']
categorical_features = ['Pclass', 'Sex', 'Embarked']

# Pipeline para las columnas numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),    # Rellenar con la mediana
    ('scaler', StandardScaler())                      # Escalar valores
])

# Pipeline para las columnas categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Rellenar con el valor más común
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # Convertir a variables dummy
])

# Combinar ambos transformadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Dividir dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Entrenamiento del modelo y predicción de resultados

In [4]:
# Crear el pipeline completo con preprocesamiento + modelo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Entrenar modelo
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)

# Evaluar rendimiento
print("Exactitud (accuracy):", accuracy_score(y_test, y_pred))
print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred))

Exactitud (accuracy): 0.7821229050279329

Matriz de confusión:
 [[84 21]
 [18 56]]

Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.82      0.80      0.81       105
           1       0.73      0.76      0.74        74

    accuracy                           0.78       179
   macro avg       0.78      0.78      0.78       179
weighted avg       0.78      0.78      0.78       179



## Guardar modelo entrenado en un nuevo archivo

In [5]:
# Guardar el modelo entrenado en un archivo .pkl
joblib.dump(model, 'titanic_model.pkl')
print("Modelo guardado correctamente")

Modelo guardado correctamente


## Comprobación de nuestro modelo por medio de un ejemplo, su probabilidad y si:(sobrevive, no sobrevive)

In [6]:
import pandas as pd
import joblib

# Cargar modelo entrenado
loaded_model = joblib.load('titanic_model.pkl')

def predecir_supervivencia(pclass, sex, age, fare, embarked):
    """
    Recibe los datos de un pasajero y devuelve la predicción:
    - 1 si sobrevive
    - 0 si no sobrevive
    Además muestra la probabilidad asociada.
    """

    # Crear un DataFrame con una sola fila (valores entre corchetes)
    input_data = pd.DataFrame({
        'Pclass': [pclass],
        'Sex': [sex],
        'Age': [age],
        'Fare': [fare],
        'Embarked': [embarked]
    })

    # Obtener la predicción y la probabilidad
    prediction = loaded_model.predict(input_data)[0]
    proba_survival = loaded_model.predict_proba(input_data)[0][1]  # probabilidad de sobrevivir

    # Traducir resultado
    resultado = "Sobrevive" if prediction == 1 else "No sobrevive"

    print(f"Predicción: {resultado}")
    print(f"Probabilidad de supervivencia: {proba_survival * 100:.2f}%")

    return prediction, proba_survival

In [7]:
predecir_supervivencia(
    pclass=1,
    sex='female',
    age=56,
    fare=7.25,
    embarked='S'
)

Predicción: Sobrevive
Probabilidad de supervivencia: 92.00%


(np.int64(1), np.float64(0.92))